# 🥈 **Silver Layer: Data Cleaning & Standardization**


The **Silver Layer** represents the "Cleaned" and "Validated" state of our data. In this stage, we transform the raw Bronze data into a queryable format by applying a strict sequence of cleaning and casting operations.

# 🚀 ETL Transformation: Stores Locations Silver Layer
**Objective:** Standardize store metadata by performing advanced string cleaning, enforcing numerical data integrity, and simplifying geographical attributes for downstream BI reporting.

---
### ✅ Summary of Transformation Steps
| Step | Operation | Tool |
| :--- | :--- | :--- |
| **1** | Environment Shift | `.toPandas()` |
| **2** | Text Trimming | `.str.strip()` |
| **3** | Case Standardization | `.str.upper()` |
| **4** | Type Conversion | `.astype('Int64')` |
| **5** | Delimiter Splitting | `.str.split().str[-1]` |
| **6** | Silver Layer Save | `spark.saveAsTable()` |

### 1. Data Acquisition & Environment Transition
The process begins by loading the raw data from the Bronze Layer. The Spark DataFrame is converted to Pandas to allow for granular string manipulations and localized data cleaning.
* **Source Table:** `sales.bronz_layer.stores_locations`
* **Method:** `.toPandas()` for high-performance text processing.

---

In [0]:
import pandas as pd 
import pyspark.sql.functions as f

stores_locations = spark.read.table("sales.bronz_layer.stores_locations")
stores_locations = stores_locations.toPandas()
stores_locations.head()


,_StoreID,City_Name,County,StateCode,State,Type,Latitude,Longitude,AreaCode,Population,Household_Income,Median_Income,Land_Area,Water_Area,Time_Zone,ingestion_data
0,1,Birmingham,Shelby County/Jefferson County,AL,Alabama,City,33.52744,-86.79905,205,212461.0,89972.0,31061.0,378353942.0,6591013.0,America/Chicago,2026-02-02 12:53:40.300038
1,2,Huntsville,Limestone County/Madison County,AL,Alabama,City,34.69901,-86.67298,256,190582.0,78554.0,48775.0,552604579.0,3452021.0,America/Chicago,2026-02-02 12:53:40.300038
2,3,Mobile,Mobile County,AL,Alabama,City,30.69436,-88.04305,251,194288.0,76170.0,38776.0,361044263.0,105325210.0,America-Chicago,2026-02-02 12:53:40.300038
3,4,Montgomery,Montgomery County,AL,Alabama,City,32.36681,-86.29997,334,200602.0,79866.0,42927.0,413985435.0,4411954.0,America/Chicago,2026-02-02 12:53:40.300038
4,5,Little Rock,Pulaski County,AR,Arkansas,City,34.74648,-92.28959,501,197992.0,79902.0,46085.0,307398785.0,6758644.0,America/Chicago,2026-02-02 12:53:40.300038


### 2. Automated Whitespace Trimming
Hidden whitespaces in string columns often lead to failed joins and incorrect aggregations in Power BI. 
* **Detection:** The script dynamically identifies all columns with `object` (string) data types.
* **Action:** Iterates through these columns using `.str.strip()` to remove leading and trailing spaces.
* **Result:** Ensures data consistency across all descriptive fields.

---

In [0]:
#identify columns contain text 
string_columns = stores_locations.select_dtypes(include =['object']).columns

#loop through and trim leading and trailing whitespaces 
for col in string_columns: 
    stores_locations[col] = stores_locations[col].astype(str).str.strip()

print(f"Trimmed {len(string_columns)} columns :{list(string_columns)}")

Trimmed 6 columns :['City_Name', 'County', 'StateCode', 'State', 'Type', 'Time_Zone']


### 3. State Code Normalization
To ensure that geographical grouping works correctly, state identifiers must follow a uniform casing standard.
* **Process:** Analyzed unique value counts before and after cleaning.
* **Action:** Applied `.str.upper()` to the `StateCode` column.
* **Impact:** Merges duplicate entries caused by casing differences (e.g., "ny" and "NY" become "NY").

---

In [0]:
print("Unique State Code ")
print(stores_locations['StateCode'].value_counts())
stores_locations['StateCode'] = stores_locations['StateCode'].str.upper()
print("Unique Currency codes after cleaning")
print(stores_locations['StateCode'].value_counts())

Unique State Code 
StateCode
CA    73
TX    36
IL    30
FL    25
NY    19
CO    13
IN    12
AZ    10
CT    10
NC     9
VA     8
WA     8
NV     8
TN     6
OH     6
NJ     6
GA     6
MI     6
MA     5
OR     5
LA     5
KS     5
AL     4
MO     4
UT     4
OK     4
MN     3
SC     3
IA     3
PA     3
NM     2
WI     2
MD     2
tx     2
NE     2
KY     2
SD     1
RI     1
MS     1
ny     1
NH     1
ND     1
MT     1
AR     1
mo     1
in     1
ID     1
HI     1
ga     1
DC     1
ca     1
wi     1
Name: count, dtype: int64
Unique Currency codes after cleaning
StateCode
CA    74
TX    38
IL    30
FL    25
NY    20
CO    13
IN    13
AZ    10
CT    10
NC     9
NV     8
VA     8
WA     8
GA     7
NJ     6
TN     6
OH     6
MI     6
MO     5
OR     5
KS     5
MA     5
LA     5
OK     4
UT     4
AL     4
PA     3
SC     3
WI     3
MN     3
IA     3
NM     2
NE     2
MD     2
KY     2
NH     1
ND     1
MT     1
AR     1
RI     1
SD     1
ID     1
HI     1
DC     1
MS     1
Name: count, dtype: int64

### 4. Robust Numerical Type Enforcement
To enable mathematical analysis, numerical columns stored as text are converted to proper integer formats.
* **Data Types:** Converted to `Int64` (Nullable Integer).
* **Benefit:** Unlike standard integers, `Int64` allows the dataset to handle missing values (`NaN`) without crashing the ETL pipeline.
* **Target Columns:** `Population`, `Household_Income`, `Median_Income`, `Land_Area`, and `Water_Area`.

---
### 5. Time Zone Simplification
Raw time zone data typically includes regional prefixes that clutter visualizations.
* **Logic:** Used `.str.split('/')` to isolate the specific location.
* **Extraction:** Retained only the final element of the path (e.g., "America/New_York" → "New_York").

---

In [0]:
#convert to integers 
integers_columns = ['Population','Household_Income','Median_Income','Land_Area','Water_Area']
stores_locations[integers_columns] = stores_locations[integers_columns].astype('Int64')
print("Information of stores_locations dataframe:")
stores_locations.info()

#Remove 'America/' prefix 
stores_locations['Time_Zone'] = stores_locations['Time_Zone'].str.split('/').str[-1]
# Check changes
print("\nFirst 5 entries of cleaned Time_Zone column:")
print(stores_locations['Time_Zone'].head(5))

Information of stores_locations dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   _StoreID          367 non-null    int64         
 1   City_Name         367 non-null    object        
 2   County            367 non-null    object        
 3   StateCode         367 non-null    object        
 4   State             367 non-null    object        
 5   Type              367 non-null    object        
 6   Latitude          367 non-null    float64       
 7   Longitude         367 non-null    float64       
 8   AreaCode          367 non-null    int64         
 9   Population        367 non-null    Int64         
 10  Household_Income  367 non-null    Int64         
 11  Median_Income     367 non-null    Int64         
 12  Land_Area         367 non-null    Int64         
 13  Water_Area        367 non-null    Int

### 6. Schema Refinement & Column Pruning
To align with the Silver Layer's "Business Ready" standards, the schema was refined for clarity.
* **Renaming:** Updated technical headers to user-friendly names (e.g., `_StoreID` → `Store_ID`).
* **Pruning:** Dropped `Latitude` and `Longitude` to reduce noise, as these coordinates are not required for high-level regional sales analysis.

---

### 7. Delta Lake Finalization
The cleaned data is converted back to a Spark DataFrame and written to the Delta Lake using a "Save and Overwrite" strategy.
* **Format:** **Delta** (Supporting ACID transactions and schema enforcement).
* **Destination:** `sales.silver_layer.stores_locations`
* **Write Mode:** `overwriteSchema = true` to ensure long-term pipeline flexibility.



---

In [0]:
#Rename Columns 
stores_locations = stores_locations.rename(
    columns= {
   '_StoreID': 'Store_ID',
   'StateCode': 'State_Code',
   'AreaCode' : 'Area_Code'
    }
)
# drop not needed columns
columns_to_drop  = ['Latitude','Longitude'] 
stores_locations = stores_locations.drop(columns_to_drop,axis=1)

#save data as spark table 
stores_locations_spark = spark.createDataFrame(stores_locations)
stores_locations_spark.write\
       .mode("overwrite")\
       .format("delta")\
       .option("overwriteSchema","true")\
       .saveAsTable("sales.silver_layer.stores_locations")

### sanity check for silver table 

In [0]:
%sql 
SELECT * FROM sales.silver_layer.stores_locations LIMIT 10 ; 

Store_ID,City_Name,County,State_Code,State,Type,Area_Code,Population,Household_Income,Median_Income,Land_Area,Water_Area,Time_Zone,ingestion_data
1,Birmingham,Shelby County/Jefferson County,AL,Alabama,City,205,212461,89972,31061,378353942,6591013,Chicago,2026-02-02T12:53:40.300Z
2,Huntsville,Limestone County/Madison County,AL,Alabama,City,256,190582,78554,48775,552604579,3452021,Chicago,2026-02-02T12:53:40.300Z
3,Mobile,Mobile County,AL,Alabama,City,251,194288,76170,38776,361044263,105325210,America-Chicago,2026-02-02T12:53:40.300Z
4,Montgomery,Montgomery County,AL,Alabama,City,334,200602,79866,42927,413985435,4411954,Chicago,2026-02-02T12:53:40.300Z
5,Little Rock,Pulaski County,AR,Arkansas,City,501,197992,79902,46085,307398785,6758644,Chicago,2026-02-02T12:53:40.300Z
6,Chandler,Maricopa County,AZ,Arizona,City,480,260828,88973,72695,168153308,260378,Phoenix,2026-02-02T12:53:40.300Z
7,Gilbert,Maricopa County,AZ,Arizona,Town,480,247542,73617,82424,176124143,510475,Phoenix,2026-02-02T12:53:40.300Z
8,Glendale,Maricopa County,AZ,Arizona,City,623,240126,79152,46776,153029354,1003996,Phoenix,2026-02-02T12:53:40.300Z
9,Mesa,Maricopa County,AZ,Arizona,City,480,471825,168914,48809,357097953,1950781,Phoenix,2026-02-02T12:53:40.300Z
10,Peoria,Maricopa County/Yavapai County,AZ,Arizona,City,928,171237,58588,65314,455011748,8864869,Phoenix,2026-02-02T12:53:40.300Z
